## Retrieval augmented generation
 
In retrieval augmented generation (RAG), an LLM retrieves contextual documents from an external dataset as part of its execution. 

This is useful if we want to ask question about specific documents (e.g., our PDFs, a set of videos, etc). 

![](overview.jpg)

# Document Loading

![](data_loading_sources.jpg)

### Loaders
* Loaders deal with the specifics of accessing and converting data
    * Accessing
        * Web Sites
        * Data Bases
        * YouTube arXiv
        * ...
    * Data Types
        * PDF
        * HTML
        * JSON
        * Word, PowerPoint...
* Returns a list of 'Document objects:

[<br>
Document (page_content='MachineLearning-Lecture01 \nInstructor (Andrew Ng): Okay.<br>
Good morning. Welcome to CS229.<br>
metadata={'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf, 'page': 0})<br>
...<br>
Document (page_content='[End of Audio] \Duration: 69 minutes<br>
metadata={'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf, 'page': 21})<br>
]

Each page is a `Document`.

A `Document` contains text (`page_content`) and `metadata`.

In [ ]:
import os
import openai

openai.api_key  = "paste-your-api-key-here"

### PDFs

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/00.pdf")
pages = loader.load()

In [ ]:
len(pages)

In [ ]:
pages[0]

In [ ]:
print(pages[0].page_content)

In [ ]:
pages[0].metadata

### YouTube

In [ ]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [ ]:
# ! pip install yt_dlp
# ! pip install pydub

**Note**: This can take several minutes to complete.

In [ ]:
url="https://www.youtube.com/watch?v=VcPMLDu7BTs"
save_dir="docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)
video_content = loader.load()

In [ ]:
video_content[0].page_content[0:500]

In [ ]:
video_content[0].metadata

### URLs

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/basecamp/handbook/blob/master/37signals-is-you.md")

In [ ]:
docs = loader.load()

In [ ]:
print(docs[0].page_content[:500])

In [ ]:
docs[0].metadata

# Document Splitting

![](document_splitting.jpg)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [ ]:
chunk_size =26
chunk_overlap = 4

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [ ]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'

In [ ]:
r_splitter.split_text(text2)

In [ ]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"

In [ ]:
r_splitter.split_text(text3)

In [ ]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)

In [ ]:
c_splitter.split_text(text3)

In [ ]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [ ]:
len(some_text)

In [ ]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
c_splitter.split_text(some_text)

In [ ]:
r_splitter.split_text(some_text)

Let's reduce the chunk size a bit and add a period to our separators:

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""],
    is_separator_regex = True
)
r_splitter.split_text(some_text)

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    is_separator_regex = True
)
r_splitter.split_text(some_text)

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    # separators=["\n\n", "\n", "(?<=\. )", " ", ""]
    separators=["\n\n", "(?<=\. )", "\n", " ", ""],
    is_separator_regex = True
)

In [ ]:
docs = r_splitter.split_documents(pages)

In [ ]:
len(docs)

In [ ]:
docs[4].page_content

In [ ]:
# filter newline chars from all the documents
for doc in docs:
    doc.page_content = doc.page_content.replace("\n", " ")

In [ ]:
docs[4].page_content

In [ ]:
# Load all PDF
loaders = [PyPDFLoader(f"docs/{file}") for file in os.listdir('docs') if file.endswith('.pdf')]

docs = []
for loader in loaders:
    pdf_docs = loader.load()
    docs += pdf_docs

In [ ]:
print(pdf_docs[0].page_content)

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150,
    separators=["\n\n", "(?<=\. )", "\n", " ", ""],
    is_separator_regex = True
)

splits = text_splitter.split_documents(docs)
for doc in splits:
    doc.page_content = doc.page_content.replace("\n", " ").replace('- ', "")
len(splits)

In [ ]:
splits[10]

# Vectorstores and Embeddings

![](embeddings.jpg)

In [ ]:
# Use this if you have paid version of OpenAI 
# from langchain.embeddings.openai import OpenAIEmbeddings
# embedding = OpenAIEmbeddings(openai_api_key="paste-your-api-key-here")

In [ ]:
# Use this if you don't have paid version of OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
model_id = 'sentence-transformers/all-mpnet-base-v2'
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(
    model_name=model_id,
    model_kwargs=model_kwargs
)

In [ ]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [ ]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [ ]:
import numpy as np

In [ ]:
np.dot(embedding1, embedding2)

In [ ]:
np.dot(embedding1, embedding3)

In [ ]:
np.dot(embedding2, embedding3)

### Vectorstores

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = 'chroma/'

In [ ]:
!rm -rf ./chroma  # remove old database files if any

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

### Similarity Search

In [ ]:
question = "How can IMBH grow in mass?"

In [ ]:
docs = vectordb.similarity_search(question, k=3)
len(docs)

In [ ]:
docs[0].page_content

In [ ]:
for doc in docs:
    print(doc.page_content, '\n')

Let's save this so we can use it later!

In [ ]:
vectordb.persist()

# Retrieval

![](overview.jpg)

Can we provide full text as a context for GPT? Estimate it!

In [ ]:
texts = [
    "I like space and IT, that's why I came to Space summer school in Kosice.",
    "Black holes are an interesting phenomenon to study.",
    "Typically, there's a supermassive black hole sitting in the middle of a galaxy.",
]

In [ ]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [ ]:
question = "Tell me something interesting about black holes."

In [ ]:
smalldb.similarity_search(question, k=2)

### Addressing Diversity: Maximum marginal relevance
 
`Maximum marginal relevance` strives to achieve both relevance to the query *and diversity* among the results.

In [ ]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3, lambda_mult=0.5)

In [ ]:
question = "What are some physical properties of IMBH?"
docs_ss = vectordb.similarity_search(question, k=3)

In [ ]:
docs_ss[0].page_content

In [ ]:
docs_ss[1].page_content

In [ ]:
docs_mmr = vectordb.max_marginal_relevance_search(question, k=3, fetch_k=10)

In [ ]:
docs_mmr[0].page_content

In [ ]:
docs_mmr[1].page_content

### Addressing Specificity: working with metadata

In last lecture, we showed that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on `metadata`.

`metadata` provides context for each embedded chunk.

In [ ]:
question = "What do they say about IMBH on the first page of the fifth document?"

In [ ]:
splits[0]

In [ ]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"docs/04.pdf", "page": 1}
)

In [ ]:
for d in docs:
    print(d.metadata)

In [ ]:
docs

### Addressing Specificity: working with metadata using self-query retriever

But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:
 
1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chat_models import ChatOpenAI

In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The scientific paper the chunk is from, should be one of `docs/00.pdf`, `docs/01.pdf`, `docs/02.pdf`, `docs/03.pdf`, `docs/04.pdf`, `docs/05.pdf`, `docs/06.pdf`, `docs/07.pdf`, `docs/08.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page of the scientific paper",
        type="integer",
    ),
]

In [ ]:
llm_name = "gpt-3.5-turbo"
document_content_description = "Scientific paper about black holes"
llm = ChatOpenAI(model_name=llm_name, temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [ ]:
question = "What do they say about IMBH on the first page of the fifth document?"

In [ ]:
docs = retriever.get_relevant_documents(question)

In [ ]:
for d in docs:
    print(d.metadata)

In [ ]:
docs

### Other types of retrieval

It's worth noting that vectordb as not the only kind of tool to retrieve documents. 

The `LangChain` retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [ ]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever

In [ ]:
svm_retriever = SVMRetriever.from_texts([el.page_content for el in splits], embedding)
tfidf_retriever = TFIDFRetriever.from_texts([el.page_content for el in splits])

In [ ]:
question = "What is a typical size of IMBH?"
docs_svm = svm_retriever.get_relevant_documents(question)
docs_svm

In [ ]:
question = "What is a typical size of IMBH?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf

# Question Answering

In [ ]:
llm_name = "gpt-3.5-turbo"

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key="paste-your-api-key-here")

### RetrievalQA chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5, "distance_metric": "cos"})

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

In [ ]:
result = qa_chain({"query": "Tell me something interesting about IMBH."})
result["result"]

In [ ]:
result = qa_chain({"query": "How do IMBH compare to super-massive black holes?"})
result["result"]

In [ ]:
result['source_documents']

In [ ]:
result = qa_chain({"query": "Which type of black holes are the most common?"})
result["result"]

In [ ]:
result = qa_chain({"query": "What is a typical density of IMBH?"})
result["result"]

In [ ]:
result = qa_chain({"query": "What is a typical size of IMBH?"})
result["result"]

In [ ]:
result = qa_chain({"query": "Do we have some IMBH in our galaxy?"})
result["result"]

### Prompt

Default prompts:
https://github.com/samrawal/langchain-prompts

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 8, "distance_metric": "cos"}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
result = qa_chain({"query": "How do IMBH compare to super-massive black holes?"})
result["result"]

### Advanced chain types - MapReduce, Refine

![overview.jpeg](map_reduce_refine.jpg)

In [ ]:
from langchain.chains.question_answering import load_qa_chain

query = "What are some methods to study the formation of IMBH?"

retriever = vectordb.as_retriever(search_kwargs={"k": 8, "distance_metric": "cos"})
docs = retriever.get_relevant_documents(query)

chain = load_qa_chain(llm, chain_type="map_reduce", return_map_steps=True)
ans = chain({"input_documents": docs, "question": query})  # , return_only_outputs=True)
ans['output_text']

In [ ]:
ans['intermediate_steps']

In [ ]:
query = "What are some methods to study the formation of IMBH?"

retriever = vectordb.as_retriever(search_kwargs={"k": 8, "distance_metric": "cos"})
docs = retriever.get_relevant_documents(query)

chain = load_qa_chain(llm, chain_type="refine", return_intermediate_steps=True)
ans = chain({"input_documents": docs, "question": query})  # , return_only_outputs=True)
ans['output_text']

In [ ]:
ans['intermediate_steps']

# Chat

Recall the overall workflow for retrieval augmented generation (RAG):

![overview.jpeg](overview.jpg)

In [ ]:
# by default, there's no memory!
retriever = vectordb.as_retriever(search_kwargs={"k": 8, "distance_metric": "cos"})

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

result = qa_chain({"query": "Tell me something interesting about IMBH."})
print(result["result"])

print("=" * 50)
result = qa_chain({"query": "Which of these theories is the most popular among scientists?"})
print(result["result"])

### Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

### ConversationalRetrievalChain

In [ ]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [ ]:
question = "Tell me something interesting about IMBH."
result = qa({"question": question})
result['answer']

In [ ]:
question = "Which of these theories is the most popular among scientists?"
result = qa({"question": question})
result['answer']